#Validating Configuration Data:


In [0]:
#Reading configuration file and validating all API keys

import json

In [0]:
with open('config.json','r') as f:
    config_data = f.read()

config = json.loads(config_data)

In [4]:
#Validating AWS Credentials
!pip install botocore
import botocore.session

access_key = config['AWS Access Keys']
secret_access_key = config['AWS Secret Keys']

session = botocore.session.get_session()

sts = session.create_client('sts', aws_access_key_id=access_key,aws_secret_access_key=secret_access_key)

response = sts.get_caller_identity()

if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('AWS Credentials are valid!')
    #Proceed to next validation
else:
    print('AWS Credentials are Invalid')
    #Exit code

    100% |████████████████████████████████| 5.1MB 7.5MB/s 
    100% |████████████████████████████████| 552kB 24.7MB/s 
AWS Credentials are valid!


In [6]:
#Validating Microsoft Azure API connections
from http.client import HTTPSConnection


uri = config['MS Azure Region']+'.api.cognitive.microsoft.com'
accessKey = config['MS Azure Access Keys']

test_doc = {'documents' : [
  {'id': '1', 'language': 'en', 'text': 'This is test data!'}
]}

def GetSentiment (documents):
    path = '/text/analytics/v2.0/sentiment'
    
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = HTTPSConnection (uri)
    body = json.dumps (documents)
    conn.request ("POST", path, body, headers)
    response = conn.getresponse ()
    return response

result = GetSentiment(test_doc)

if result.status == 200:
    print('Microsoft Keys are valid!')
    #Proceed with other validation
else :
    print('Microsoft Keys are Invalid!')
    #Exit code

Microsoft Keys are valid!


In [10]:
#Validating IBM API keys

#Introducing IBM API:
!pip install watson_developer_cloud
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    iam_apikey=config['IBM API Key'],
    url=config['IBM API url']
)
response = natural_language_understanding.analyze(
    text='This is test text',
    features=Features(sentiment=SentimentOptions(targets=[''])))

if response.get_status_code() == 200:
    print('IBM keys are valid!')
    #Proceed with other validation
else :
    print('IBM Keys are Invalid!')
    #Exit code

IBM keys are valid!


In [23]:
#Valdating Google API Keys:

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from google.oauth2 import service_account

cred = service_account.Credentials.from_service_account_file('NLP-Sentiment-3ab1018e6990.json')

client = language.LanguageServiceClient(credentials=cred)

data = ['This is the best day of my life!']

for text in data:
  document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)
  sentiment = client.analyze_sentiment(document=document)


  if sentiment.language == "en":
    print("Google API Keys are valid!")
  else:
    print("Invalid Google API Keys!")

Google API Keys are valid!


In [0]:
#Here we will merge all the data from all the team to get the complete dataset with manual sentiment analysis:

with open('data.json','r') as f:
    data_all = f.read()
    
data = json.loads(data_all)

In [0]:
#This is the list of Text
X = list(data['Text'].values())
#This is the list of Manual Sentiment
Y = list(data['Sentiment'].values())
#This is the list for Company name
Z = list(data['Company'].values())

In [0]:
import pandas as pd

In [0]:
Man_Senti = pd.DataFrame({'Text':X, 'Sentiment': Y, 'Company': Z})

In [80]:
Man_Senti.head()

,Company,Sentiment,Text
0,Amazon,Neutral,"Amazon.com, Inc. (NASDAQ:AMZN) Q3 2018 Earning..."
1,Amazon,Neutral,"Good day, everyone, and welcome to the Amazon...."
2,Amazon,Neutral,"For opening remarks, I will be turning the cal..."
3,Amazon,Neutral,"Hello, and welcome to our Q3 2018 financial re..."
4,Amazon,Neutral,Our comments and responses to your questions r...


In [109]:
len(Man_Senti)

622

In [82]:
text = X

from time import time
!pip install boto3
import boto3

In [0]:
#Getting sentiment using AWS API:

start = time()

comprehend = boto3.client(
    'comprehend',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_access_key,
    region_name='us-east-1'
)

rows = []
i=1
for t in text:
  sentimentData = comprehend.detect_sentiment(Text=t, LanguageCode='en')
  row = [i, 'AWS', sentimentData['Sentiment'],sentimentData['SentimentScore']['Negative'],sentimentData['SentimentScore']['Neutral'],sentimentData['SentimentScore']['Positive']]
  rows.append(row)
  i+=1
result_aws = pd.DataFrame(rows, columns=['Text_ID', 'API', 'Overall', 'Negative', 'Neutral', 'Positive'])

end = time()

time_aws = end - start


In [85]:
#AWS Sentiment 
result_aws.head()

,Text_ID,API,Overall,Negative,Neutral,Positive
0,1,AWS,NEUTRAL,0.000141,0.998418,0.001304
1,2,AWS,POSITIVE,0.003760,0.487941,0.495716
2,3,AWS,NEUTRAL,0.007338,0.976732,0.012431
3,4,AWS,NEUTRAL,0.000654,0.686703,0.305323
4,5,AWS,NEUTRAL,0.002042,0.996204,0.001183


In [86]:
time_aws

62.816622495651245

In [94]:
len(result_aws)

622

In [0]:
#Getting sentiment using Microsoft Azure API:

n = len(text)
id=[i for i in range(1,1+n)]
language=['en' for i in range(n)]
document=[]

#Generating form for hitting it on Azure Text Analystics API 
for i in range(n):
    dic = {'id':id[i],'language':language[i],'text':text[i]}
    document.append(dic)
documents = {'documents':document}

def GetSentiment (documents1):
    path = '/text/analytics/v2.0/sentiment'
    
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = HTTPSConnection (uri)
    body = json.dumps (documents1)
    conn.request ("POST", path, body, headers)
    response = conn.getresponse ()
    return response.read ()
  
start = time()
result = GetSentiment(documents)

sentiments = json.loads(result)["documents"]

i = 0
rows = []
for item in sentiments:
  row = [sentiments[i]['id'], 'Azure', sentiments[i]['score']]
  rows.append(row)
  i+=1
result_azure = pd.DataFrame(rows, columns=['Text_ID', 'API', 'Overall_Score'])

end = time()

time_azure = end - start

In [89]:
result_azure.head()

,Text_ID,API,Overall_Score
0,1,Azure,0.500000
1,2,Azure,0.891742
2,3,Azure,0.500000
3,4,Azure,0.500000
4,5,Azure,0.500000


In [90]:
time_azure

3.167090654373169

In [92]:
len(result_azure)

622

In [95]:
#Getting sentiment using IBM API:

#Introducing IBM API:
!pip install watson_developer_cloud
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    iam_apikey=config['IBM API Key'],#need private key here
    url=config['IBM API url']
)

start = time()
rows = []
i=1
for t in text:
    if (len(t)<15 or len(t.split(' '))<3):
        #Padding neutral text as the IBM NLU api needs some minimum text word and charachater coun
        t = t + "This is neutral"
    sentimentData = natural_language_understanding.analyze(text=t,features=Features(sentiment=SentimentOptions(targets=['']))).get_result()
    row = [i, 'IBM', sentimentData['sentiment']['document']['label'],sentimentData['sentiment']['document']['score']]
    rows.append(row)
    i+=1
    
result_ibm = pd.DataFrame(rows, columns=['Text_ID', 'API', 'Overall', 'Score'])

end = time()

time_ibm = end - start

In [96]:
result_ibm.head()

,Text_ID,API,Overall,Score
0,1,IBM,neutral,0.000000
1,2,IBM,positive,0.750129
2,3,IBM,positive,0.476440
3,4,IBM,positive,0.678515
4,5,IBM,positive,0.437226


In [97]:
time_ibm

229.88341784477234

In [98]:
len(result_ibm)

622

In [0]:
#Getting Sentiment using Google API:

start = time()

rows = []
i=1

for t in text:
  document = types.Document(
    content=t,
    type=enums.Document.Type.PLAIN_TEXT)
  sentiment = client.analyze_sentiment(document=document)
  row = [i, 'GCP', sentiment.document_sentiment.magnitude,sentiment.document_sentiment.score]
  rows.append(row)
  i+=1

result_gcp = pd.DataFrame(rows, columns=['Text_ID', 'API', 'Magnitude', 'Score'])

end = time()

time_gcp = end-start
  

In [100]:
result_gcp.head()

,Text_ID,API,Magnitude,Score
0,1,GCP,0.0,0.0
1,2,GCP,0.9,0.1
2,3,GCP,0.0,0.0
3,4,GCP,1.2,0.3
4,5,GCP,0.2,0.0


In [101]:
time_gcp

47.409239530563354

In [102]:
len(result_gcp)

622

In [104]:
#Getting Sentiment using Vedor API:

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()

start = time()

rows = []
i=1
for t in text:
  sentimentData = analyzer.polarity_scores(t)
  row = [i, 'Vader', sentimentData['compound'],sentimentData['neg'],sentimentData['neu'],sentimentData['pos']]
  rows.append(row)
  i+=1
  
result_vader = pd.DataFrame(rows, columns=['Text_ID', 'API', 'Overall_Score', 'Negative_Score', 'Neutral_Score', 'Positive_Score'])

end = time()

time_vedar = end - start



    100% |████████████████████████████████| 133kB 6.0MB/s 


In [105]:
result_vader.head()

,Text_ID,API,Overall_Score,Negative_Score,Neutral_Score,Positive_Score
0,1,Vader,0.0000,0.0,1.000,0.000
1,2,Vader,0.7096,0.0,0.848,0.152
2,3,Vader,0.3182,0.0,0.892,0.108
3,4,Vader,0.8658,0.0,0.872,0.128
4,5,Vader,0.0000,0.0,1.000,0.000


In [106]:
time_vedar

0.7608842849731445

In [110]:
len(result_vader)

622

In [111]:
Man_Senti.head()

,Company,Sentiment,Text
0,Amazon,Neutral,"Amazon.com, Inc. (NASDAQ:AMZN) Q3 2018 Earning..."
1,Amazon,Neutral,"Good day, everyone, and welcome to the Amazon...."
2,Amazon,Neutral,"For opening remarks, I will be turning the cal..."
3,Amazon,Neutral,"Hello, and welcome to our Q3 2018 financial re..."
4,Amazon,Neutral,Our comments and responses to your questions r...


In [0]:
Man_Senti = Man_Senti.reset_index()

In [113]:
Man_Senti.head()

,index,Company,Sentiment,Text
0,0,Amazon,Neutral,"Amazon.com, Inc. (NASDAQ:AMZN) Q3 2018 Earning..."
1,1,Amazon,Neutral,"Good day, everyone, and welcome to the Amazon...."
2,2,Amazon,Neutral,"For opening remarks, I will be turning the cal..."
3,3,Amazon,Neutral,"Hello, and welcome to our Q3 2018 financial re..."
4,4,Amazon,Neutral,Our comments and responses to your questions r...


In [0]:
Man_Senti['Text_ID'] = Man_Senti.index + 1

In [115]:
Man_Senti.head()

,index,Company,Sentiment,Text,Text_ID
0,0,Amazon,Neutral,"Amazon.com, Inc. (NASDAQ:AMZN) Q3 2018 Earning...",1
1,1,Amazon,Neutral,"Good day, everyone, and welcome to the Amazon....",2
2,2,Amazon,Neutral,"For opening remarks, I will be turning the cal...",3
3,3,Amazon,Neutral,"Hello, and welcome to our Q3 2018 financial re...",4
4,4,Amazon,Neutral,Our comments and responses to your questions r...,5


In [0]:
Man_Senti = Man_Senti[['Text_ID','Company','Text','Sentiment']]

In [118]:
Man_Senti.head()

,Text_ID,Company,Text,Sentiment
0,1,Amazon,"Amazon.com, Inc. (NASDAQ:AMZN) Q3 2018 Earning...",Neutral
1,2,Amazon,"Good day, everyone, and welcome to the Amazon....",Neutral
2,3,Amazon,"For opening remarks, I will be turning the cal...",Neutral
3,4,Amazon,"Hello, and welcome to our Q3 2018 financial re...",Neutral
4,5,Amazon,Our comments and responses to your questions r...,Neutral


In [0]:
#Now we will normalize the different API Results:

#1. AWS

